## Bayesian Modeling with CUQIpy

In this notebook we show how to use CUQIpy for constructing Bayesian models and running inference on them.

**This functionality is WIP and the user API may change in future releases.**

### Comment on purpose for the design choices in CUQIpy
We aim to provide a framework for Bayesian inference that allows rapid prototyping of Bayesian models.
In particular, it should be easy to construct or modify a Bayesian models following e.g. the principles of
*Bayesian workflow* as described in [1].

It should also be easy to change components of the sampling strategy, allowing the user to optimize the sampling for their specific problem.


### References
[1] *Gelman, A. et. al. 2020. Bayesian Workflow. arXiv:2011.01808*

[2] *Bardsley, J. 2018. Computational Uncertainty Quantification for Inverse Problems. SIAM, Society for Industrial and Applied Mathematics.*

First we import the necessary packages.

Note we require the following extra packages:
- pytorch
- pyro (If python 3.10, might have to get the -dev version of pyro)
- astra (only for the 2D CT example)

In [ ]:
import sys; sys.path.append("..")
import numpy as np
import torch as xp
import scipy.io as spio
import scipy.sparse as sps
import matplotlib.pyplot as plt
import cuqi

from cuqi.distribution import JointDistribution
from cuqi.model import LinearModel
from cuqi.sampler import Gibbs, Conjugate, ConjugateApprox

from cuqipy_pytorch.distribution import Gaussian, Gamma, Lognormal
from cuqipy_pytorch.sampler import NUTS


The following are a few "hacks" to make the notebook more smooth. These will be removed in the future.

In [ ]:
# A convenience function to sample a Bayesian model
def sample_posterior(*densities, **data):
    """ Sample posterior given by a list of densities. The observations are given as keyword arguments. """
    P = JointDistribution(*densities)
    return NUTS(P(**data)).sample(500, 500)

For simplicity let us focus on a simple 1D deconvolution problem. We then aim to implement a Hierarchical model similar to the one described in [2, Chap. 5].

That is, suppose we are interested in the inverse problem

$$ \mathbf{y} = \mathbf{A}\mathbf{x} $$

where $\mathbf{A}\in\mathbb{R}^{m\times n}$ is the convolution matrix, $\mathbf{x}\in\mathbb{R}^n$ is the true signal and $\mathbf{y}\in\mathbb{R}^m$ is the convolved signal. 

The aim is to estimate the the true signal (including estimates of uncertainty) from noisy measurements $\mathbf{y}^{obs}$.

In this case, let us load the observed data and convolution matrix from a .mat file exported from Matlab code provided in [2]. We also define some helper matrices related to defining a the Gaussian prior on the signal.

This is NOT CUQIpy related code, and is only used to load the data.

In [ ]:
mat = spio.loadmat('OneDBlurGibbs.mat')               # Load example from [2]

A = xp.asarray(mat['A'], dtype=xp.float32)               # Convolution matrix
y_obs = xp.asarray(mat['b'].flatten(), dtype=xp.float32) # Observed data
x_true = xp.asarray(mat['x_true'], dtype=xp.float32)     # True solution
n,m = A.shape

# Precision and covariance matrix for a GMRF prior
L = sps.spdiags([-np.ones(n), 2*np.ones(n), -np.ones(n)], [-1, 0, 1], n, n)
C = xp.asarray(np.linalg.inv(L.todense()), dtype=xp.float32)
I = sps.eye(m)

# Plot the observed data and the true solution
plt.plot(y_obs, label='Observed data')
plt.plot(x_true, label='True solution')
plt.legend()
plt.show()


### A basic Bayesian model

Let us now define a simple Bayesian model with assuming the observed data is corrupted by Gaussian noise and assuming a Gaussian (GMRF) prior on the signal. That is assume the following

$$
\begin{align*}
    \mathbf{x} &\sim \mathcal{N}(\mathbf{0},\delta^{-1}\mathbf{C})\\
    \mathbf{y} &\sim \mathcal{N}(\mathbf{A}\mathbf{x},\lambda^{-1}\mathbf{I}_m),
\end{align*}
$$

where $\delta$ and $\lambda$ are assumed fixed (known) parameters for now. We can define these two distributions as follows.

In [ ]:
λ = 6       # Noise precision
δ = 0.04    # Prior precision

x = Gaussian(xp.zeros(n), 1/δ*C)
y = Gaussian(lambda x: A@x, 1/λ)

The above code follows closely the mathematical notation. We use the notation `.rv` to create a random variable from a distribution, and can use these when defining new random variables by given them as arguments to the distributions.

After defining the Bayesian model we can use the convenience function `sample_posterior` to sample the posterior distribution given some observations of one or more of our random variables. In this case we assume we have observed the signal $\mathbf{y}^{obs}$.

That is we can sample from

$$

\pi(\mathbf{x}\mid \mathbf{y}=\mathbf{y}^{obs})

$$

as follows



In [ ]:
samples = sample_posterior(y, x, y=y_obs)

After sampling we can visualize the samples of the posterior by e.g. looking at the sample mean and credibility interval comparing with the true signal.

In [ ]:
samples["x"].plot_ci(exact=x_true)

### Extending the Bayesian model

Suppose we did not know good values for the noise and prior precisions $\delta$ and $\lambda$. The Bayesian approach would be to include these in the model as unknowns by putting priors on them. This is where the new modelling framework in CUQIpy becomes really useful!

 Suppose now we want to extend the Bayesian model from before into a Hierarchical model similar to the one suggested in [2]. That is:

$$
\begin{align*}
    \delta &\sim \mathrm{Gamma}(1, 10^{-4})\\
    \lambda &\sim \mathrm{Gamma}(1, 10^{-4})\\
    \mathbf{x} &\sim \mathcal{N}(\mathbf{0},\delta^{-1}\mathbf{C})\\
    \mathbf{y} &\sim \mathcal{N}(\mathbf{A}\mathbf{x},\lambda^{-1}\mathbf{I}_m),
\end{align*}
$$

All we have to do is add exactly these two extra assumptions to the definition of the Bayesian model and run the posterior sampling again.

In [ ]:
δ = Gamma(1, 1e-4)
λ = Gamma(1, 1e-4)
x = Gaussian(xp.zeros(n), lambda δ: 1/δ*C)
y = Gaussian(lambda x: A@x, lambda λ: 1/λ)

samples = sample_posterior(y, x, λ, δ, y=y_obs)

Because this is a more complex Bayesian model the default sampling approach can take a while (about 15 minutes for 500 warmup samples and 5 minutes for 500 posterior samples).

As we will see later we can do a number of things to optimize the posterior sampling if we are willing to spend more effort when defining our model. On the other hand, the default sampling approach works well for a very large set of problems given enough compute time.

We can also visualize the posterior samples for each of the sampled parameters. Here we do a *trace plot* for the hyper-parameters.

In [ ]:
samples["x"].plot_ci(exact=x_true)
samples["δ"].plot_trace()
samples["λ"].plot_trace();

Before we go further with optimizing the sampling, let us dive a just little deeper into the basic primitives of CUQIpy to see how they work.

### Posterior sampling

The main objects related to posterior sampling in CUQIpy are the `JointDistribution` and `Sampler` classes.

The `JointDistribution` is responsible for defining the joint probability density function of a list of random variables. It allows computing any combination of conditional probabilities possible.

In the case below we define `J` as the joint distribution $\pi(\mathbf{x},\mathbf{y})$ and then use it to define `P` as the posterior $\pi(\mathbf{x} \mid \mathbf{y}=\mathbf{y}^{obs})$ in the line below.

Finally we need to decide on a sampling strategy for the posterior. Here we use the No-U-Turn Sampler (NUTS) which is a highly optimized Markov Chain Monte Carlo (MCMC) sampling algorithm. This is done in two stages. First we define the `sampler` object, and then we run the sampling

In [ ]:
λ = 6       # Noise precision
δ = 0.04    # Prior precision
x = Gaussian(xp.zeros(n), 1/δ*C)
y = Gaussian(lambda x: A@x, 1/λ)

# This is what happens the `sample_posterior` function
J = JointDistribution(y, x)            # Define joint distribution p(x,y)
P = J(y=y_obs)                           # Define posterior distribution p(x|y=y_obs)
sampler = NUTS(P)                        # Define sampling strategy for posterior
samples = sampler.sample(N=500, Nb=500)  # Sample from posterior

## Bonus 2: Eight schools model

In [ ]:
# Some hacks to look at the interface options
class Normal(Gaussian): pass
class LogNormal(Lognormal): pass

The eight schools model is a classic example made famous by the Bayesian Data Analysis book by Gelman et. al. 

It is often used to illustrate the notation and code-style of probabilistic programming languages. 

The actual model is explained in the BDA book or in the Edward 1.0 PPL notebook ([link](https://github.com/blei-lab/edward/blob/master/notebooks/eight_schools.ipynb)).

The Bayesian model can be written as

\begin{align*}
    \mu &\sim \mathcal{N}(0, 10^2)\\
    \tau &\sim \log\mathcal{N}(5, 1)\\
    \boldsymbol \theta' &\sim \mathcal{N}(\mathbf{0}, \mathbf{I}_m)\\
    \boldsymbol \theta &= \mu + \tau \boldsymbol \theta'\\
    \mathbf{y} &\sim \mathcal{N}(\boldsymbol \theta, \boldsymbol \sigma^2 \mathbf{I}_m)
\end{align*}

where $\mathbf{y}\in\mathbb{R}^m$ and $\boldsymbol \sigma\in\mathbb{R}^m$ is observed data.

In CUQIpy we can define the model as follows:

In [ ]:
y_obs = xp.tensor([28, 8, -3,  7, -1, 1,  18, 12], dtype=xp.float32)
σ_obs = xp.tensor([15, 10, 16, 11, 9, 11, 10, 18], dtype=xp.float32)

μ     = Normal(0, 10**2)
τ     = LogNormal(5, 1)
θp    = Normal(xp.zeros(8), 1)
θ     = lambda μ, τ, θp: μ+τ*θp
y     = Normal(θ, cov=σ_obs**2)

samples = sample_posterior(μ, τ, θp, y, y=y_obs)

In [ ]:
# Plot posterior samples
samples["θp"].plot_violin(); 
print(samples["μ"].mean()) # Average effect
print(samples["τ"].mean()) # Average variance

In [ ]:
# Plot treatment effect distribution
θs = []
for μs, τs, θps in zip(samples["μ"], samples["τ"], samples["θp"]):
    θs.append(θ(μs, τs, θps))
    
θs = cuqi.samples.Samples(xp.tensor(θs).T)
θs.geometry._name = "θ"
θs.plot_violin();